# Chapter 7

# Data Cleaning and Preparation

In [3]:
import pandas as pd

In [4]:
import numpy as np

# 7.1 Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.
The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [5]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [6]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [7]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by refer‐ ring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

In [8]:
# The built-in Python None value is also treated as NA in object arrays:
string_data[0] = None

In [9]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

# Filtering Out Missing Data

In [11]:
from numpy import nan as NA

In [12]:
data = pd.Series([1,NA,3.5,NA,7])

In [13]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [14]:
# This is equivalent to:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [15]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                             [NA, NA, NA], [NA, 6.5, 3.]])

In [16]:
cleaned = data.dropna()

In [17]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [18]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [19]:
# Passing how='all' will only drop rows that are all NA:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [20]:
# To drop columns in the same way, pass axis=1:
data[4] = NA

In [21]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [22]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the thresh argument:

In [23]:
df = pd.DataFrame(np.random.randn(7,3))

In [24]:
df.iloc[:4,1] = NA

In [25]:
df.iloc[:2,2] = NA

In [26]:
df 

,0,1,2
0,-0.930151,NaN,NaN
1,0.007112,NaN,NaN
2,2.495879,NaN,-0.840358
3,-0.003990,NaN,0.675950
4,-0.472073,-0.188372,-0.003840
5,1.229392,-0.671942,1.014626
6,-1.420488,0.711286,2.150086


In [27]:
df.dropna()

,0,1,2
4,-0.472073,-0.188372,-0.003840
5,1.229392,-0.671942,1.014626
6,-1.420488,0.711286,2.150086


In [29]:
df.dropna(thresh=2)

,0,1,2
2,2.495879,NaN,-0.840358
3,-0.003990,NaN,0.675950
4,-0.472073,-0.188372,-0.003840
5,1.229392,-0.671942,1.014626
6,-1.420488,0.711286,2.150086


# Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most pur‐ poses, the fillna method is the workhorse function to use. Calling fillna with a constant replaces missing values with that value:

In [30]:
df.fillna(0)

,0,1,2
0,-0.930151,0.000000,0.000000
1,0.007112,0.000000,0.000000
2,2.495879,0.000000,-0.840358
3,-0.003990,0.000000,0.675950
4,-0.472073,-0.188372,-0.003840
5,1.229392,-0.671942,1.014626
6,-1.420488,0.711286,2.150086


In [31]:
# Calling fillna with a dict, you can use a different fill value for each column:
df.fillna({1:0.5,2:0})

,0,1,2
0,-0.930151,0.500000,0.000000
1,0.007112,0.500000,0.000000
2,2.495879,0.500000,-0.840358
3,-0.003990,0.500000,0.675950
4,-0.472073,-0.188372,-0.003840
5,1.229392,-0.671942,1.014626
6,-1.420488,0.711286,2.150086


In [32]:
# fillna returns a new object, but you can modify the existing object in-place:
_ = df.fillna(0, inplace=True)

In [33]:
df

,0,1,2
0,-0.930151,0.000000,0.000000
1,0.007112,0.000000,0.000000
2,2.495879,0.000000,-0.840358
3,-0.003990,0.000000,0.675950
4,-0.472073,-0.188372,-0.003840
5,1.229392,-0.671942,1.014626
6,-1.420488,0.711286,2.150086


In [35]:
# The same interpolation methods available for reindexing can be used with fillna:
df = pd.DataFrame(np.random.randn(6,3))

In [36]:
df.iloc[2:,1] = NA

In [38]:
df.iloc[4:,2] = NA

In [39]:
df

,0,1,2
0,1.092856,2.170698,-0.947589
1,1.050027,-0.579414,0.026706
2,1.103138,NaN,-0.111286
3,1.389473,NaN,0.158732
4,0.616253,NaN,NaN
5,0.179656,NaN,NaN


In [40]:
df.fillna(method='ffill')

,0,1,2
0,1.092856,2.170698,-0.947589
1,1.050027,-0.579414,0.026706
2,1.103138,-0.579414,-0.111286
3,1.389473,-0.579414,0.158732
4,0.616253,-0.579414,0.158732
5,0.179656,-0.579414,0.158732


In [41]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,1.092856,2.170698,-0.947589
1,1.050027,-0.579414,0.026706
2,1.103138,-0.579414,-0.111286
3,1.389473,-0.579414,0.158732
4,0.616253,NaN,0.158732
5,0.179656,NaN,0.158732


With fillna you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [42]:
data = pd.Series([1.,NA,3.5,NA,7])

In [44]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

# 7.2 Data Transformation

So far in this chapter we’ve been concerned with rearranging data. Filtering, cleaning, and other transformations are another class of important operations.

# Removing Duplicates

In [45]:
# Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                             'k2': [1, 1, 2, 3, 3, 4, 4]})


In [46]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [47]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [48]:
# Relatedly, drop_duplicates returns a DataFrame where the duplicated array is False:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [49]:
data['v1'] = range(7)

In [50]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated and drop_duplicates by default keep the first observed value combina‐ tion. Passing keep='last' will return the last one:

In [51]:
data.drop_duplicates(['k1','k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


# Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the val‐ ues in an array, Series, or column in a DataFrame. Consider the following hypotheti‐ cal data collected about various kinds of meat:

In [52]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
         'Pastrami', 'corned beef', 'Bacon',
         'pastrami', 'honey ham', 'nova lox'],
         'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [53]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [54]:
 meat_to_animal = {
      'bacon': 'pig',
      'pulled pork': 'pig',
      'pastrami': 'cow',
      'corned beef': 'cow',
      'honey ham': 'pig',
      'nova lox': 'salmon'
}

In [55]:
lowercased = data['food'].str.lower()

In [56]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [57]:
data['animal'] = lowercased.map(meat_to_animal)

In [58]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [60]:
# We could also have passed a function that does all the work:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.

# Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement. As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so. Let’s con‐ sider this Series:

In [62]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [63]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series (unless you pass inplace=True):

In [64]:
data.replace(-999,np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value: